This notebook takes part of the "Multimodhal" project. The final goal is to use the Circular Inference model to modelize the neural pathway of auditory hallucinations in Schizophrenia's patients.

The overall aim of the code here is to provide a robust and replicable pipeline for pre-processing and reconstruction of Diffusion MRI data. X subjects have been integrated in the process. 

The code for each part, pre-processing and reconstruction, is distinct (e.g; each code part will be in its correspondnig titled part) to make it as clear as possible. For both we will use the QSIprep library (more infos here https://qsiprep.readthedocs.io/en/latest/index.html)

Principal Investigator : Renaud Jardri
Authors : Alexis Melot, Benoit Noémie
Laboratory : Lille Neuroscience & Cognition
Date : 26/01/2026


## Run QSIprep on Multimodhal data

write short description of project, and the purpose of this notebook
Authors 
date

TODO list: 
- store log into .txt file for further inspection and saving errors 


In [ ]:
import os
import subprocess
import datetime

SUBJECT = "02" # "01 02 03"
DATA_FOLDER = "~/Documents/AlexisBenoit/multimodhal_bids/bids/"
OUT_FOLDER = "~/Documents/AlexisBenoit/multimodhal_bids/results6/"
FS_LICENSE_PATH = "~/Documents/AlexisBenoit/multimodhal_bids/license.txt"

dwi_path='~/Documents/AlexisBenoit/multimodhal_bids/bids/sub-01/ses-pre/dwi/sub-01_ses-pre_dwi.nii.gz'

json_file = '/home/calculateur2/Documents/AlexisBenoit/multimodhal_bids/bids/sub-01/ses-pre/dwi/sub-01_ses-pre_dwi.json'
outputACQ = '/home/calculateur2/Documents/AlexisBenoit/multimodhal_bids/synb0/acqparams.txt'

synb0_dir="/home/calculateur2/Documents/AlexisBenoit/multimodhal_bids/synb0"
dwi_json_path="/home/calculateur2/Documents/AlexisBenoit/multimodhal_bids/bids/sub-01/ses-pre/dwi/sub-01_ses-pre_dwi.json"
fmap_json_path="/home/calculateur2/Documents/AlexisBenoit/multimodhal_bids/bids/sub-01/ses-pre/fmap/sub-01_ses-pre_dir-PA_epi.json"
b0_fmap_path="/home/calculateur2/Documents/AlexisBenoit/multimodhal_bids/bids/sub-01/ses-pre/fmap/sub-01_ses-pre_dir-PA_epi.nii.gz"


### SynB0-DISCO requires three specific files per subject to work properly:

- b0.nii.gz: The b=0 image extracted from the DWI (using nibabel).
- T1.nii.gz: The anatomical T1-weighted image.
- acqparams.txt: A text file describing the acquisition sequence.


In [ ]:
# Extracting b0.nii.gz file

import nibabel as nib 

nii_img  = nib.load(dwi_path) #load image (4D) [X,Y,Z_slice,gradient_direction] Question chemin
nii_b0 = nii_img.slicer[...,0]  # b=0 volume
nib.save(nii_b0, "~/Documents/AlexisBenoit/multimodhal_bids/synb0/b0.nii.gz" ) # save b0 image as nifti

In [ ]:
# Copying T1w image into synB0 folder 
import os
source = "/home/calculateur2/Documents/AlexisBenoit/multimodhal_bids/bids/sub-01/ses-pre/anat/sub-01_ses-pre_T1w.nii.gz"
dest = "/home/calculateur2/Documents/AlexisBenoit/multimodhal_bids/synb0/T1.nii.gz"

os.system(f'cp {source} {dest}')

In [ ]:
# #first run Synb0-disco
os.system('docker run --rm \
-v /home/calculateur2/Documents/AlexisBenoit/multimodhal_bids/synb0/:/INPUTS/ \
-v /home/calculateur2/Documents/AlexisBenoit/multimodhal_bids/synb0/outputs:/OUTPUTS/ \
-v /home/calculateur2/Documents/AlexisBenoit/multimodhal_bids/license.txt:/extra/freesurfer/license.txt \
--user $(id -u):$(id -g) \
leonyichencai/synb0-disco:v3.1 \
--no-topup')


### Synb0 to fmap dir

- b0.JSON file edition in the fmap dir
- copy of .nii.gz in the fmap dir

In [ ]:
import json
import os

# Load DWI JSON sidecar
with open(dwi_json_path, 'r') as json_file:
    dwi_json = json.load(json_file)

dwi_json['B0FieldIdentifier'] = 'synb0'

# Save JSON to dwi_json_path
with open(dwi_json_path, 'w') as json_file:
    json.dump(dwi_json, json_file, indent=4)

# Update fields
dwi_json['PhaseEncodingDirection'] = 'i-'
dwi_json['TotalReadoutTime'] = 0.01
dwi_json['EffectiveEchoSpacing'] = 0.00001
dwi_json['IntendedFor'] = os.path.abspath(dwi_json_path).replace('\\', '/').split('bids/')[-1].replace('.json', '.nii.gz')


# Save updated JSON sidecar at new_json_path
with open(fmap_json_path, 'w') as json_file:
    json.dump(dwi_json, json_file, indent=4)


In [4]:
# copy b0 undistorted image to fmap dir
os.system(f'cp {synb0_dir}'+'/outputs/b0_all_topup.nii.gz '+f'{b0_fmap_path}')

0

### Bash command line of QSIprep, integrating synb0 outputs : 2 files (.JSON and .nii.gz) of the fmap directory

In [ ]:
os.system(f'docker run --rm \
-v {DATA_FOLDER}:/mnt/rawdata \
-v {OUT_FOLDER}:/mnt/work \
-v ~/Documents/AlexisBenoit/multimodhal_bids/license.txt:/mnt/license.txt \
pennlinc/qsiprep:1.1.1 \
--output-resolution 2 \
/mnt/rawdata /mnt/work/output \
participant \
--participant-label 01 02 03 \
--fs-license-file /mnt/license.txt \
--omp-nthreads 24 \
--mem 32000 \
--skip-bids-validation \
--unringing-method mrdegibbs \
--force-syn \
--work-dir /mnt/work/workdir \
-vv \
--session-id ses-pre \
--stop-on-first-crash')

Traceback (most recent call last):
  File "/opt/conda/envs/qsiprep/bin/qsiprep", line 3, in <module>
    from qsiprep.cli.run import main
  File "/opt/conda/envs/qsiprep/lib/python3.10/site-packages/qsiprep/cli/run.py", line 27, in <module>
    from .. import config
  File "/opt/conda/envs/qsiprep/lib/python3.10/site-packages/qsiprep/config.py", line 96, in <module>
    from templateflow.conf import TF_LAYOUT
  File "/opt/conda/envs/qsiprep/lib/python3.10/site-packages/templateflow/__init__.py", line 18, in <module>
    from . import api
  File "/opt/conda/envs/qsiprep/lib/python3.10/site-packages/templateflow/api.py", line 8, in <module>
    from .conf import TF_LAYOUT, TF_S3_ROOT, TF_USE_DATALAD, requires_layout
  File "/opt/conda/envs/qsiprep/lib/python3.10/site-packages/templateflow/conf/__init__.py", line 167, in <module>
    init_layout()
  File "/opt/conda/envs/qsiprep/lib/python3.10/site-packages/templateflow/conf/__init__.py", line 151, in init_layout
    TF_LAYOUT = Layout(
 

2

In [ ]:

# # Store the logs for each pre-processing subject 
# timestamp = datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
# logstxt = f"qsiprep_log_{timestamp}.txt"

# try: 
#     process = subprocess.run(command, stdout=logstxt, stderr=subprocess.STDOUT, text=True, check=True)
# except:
#     print("Error")



Error


In [ ]:
docker run --rm \
-v ~/Documents/AlexisBenoit/multimodhal_bids/bids/:/mnt/rawdata \
-v ~/Documents/AlexisBenoit/multimodhal_bids/results/synb0qsiprep_sub1/:/mnt/work \
-v ~/Documents/AlexisBenoit/multimodhal_bids/license.txt:/mnt/license.txt \
pennlinc/qsiprep:1.1.1 \
--output-resolution 2 \
/mnt/rawdata /mnt/work/output \
participant \
--participant-label 01 \
--fs-license-file /mnt/license.txt \
--omp-nthreads 24 \
--mem 32000 \
--skip-bids-validation \
--unringing-method mrdegibbs \
--work-dir /mnt/work/workdir \
-vv \
--session-id ses-pre \
--stop-on-first-crash

SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (1095689121.py, line 9)